In [0]:
# Coder: Vahid Ghafourian
# Not Complete

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
cd gdrive/My\ Drive/Vahid\ Ghafourian/Digi2nd/

### Lets do it:

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as tts
import matplotlib.pyplot as plt
from keras.layers import *
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.models import Sequential, Model

In [0]:
digiData = pd.read_csv("balance_classified_train.csv")
digiData = digiData.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
digiValid = pd.read_csv("balance_classified_valid.csv")
digiData[:2]

In [0]:
like = digiData[digiData["target"]==(0 or 2) ]
verylike = digiData[digiData["target"]==(1 or 3) ]
xtrain = verylike.comment
likes = like.comment

In [0]:
vocab_size = 10000
embedding_dim = 120
max_length = 120

In [0]:
# Tokenizing
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(xtrain)
word_index = tokenizer.word_index

def encode(valid):
    valid_sequences = tokenizer.texts_to_sequences(valid)
    valid_data = pad_sequences(valid_sequences, padding = "post", maxlen = max_length)
    return valid_data

# get only the top frequent words on train
train_data = encode(xtrain)
ganTrain = encode(likes)
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

# print(decode_review(train_data[1]))
# print(ganTrain[1])
# print(ganTrain[1])

In [0]:
train_data = train_data.reshape((train_data.shape[0],train_data.shape[1], 1))
ganTrain = ganTrain.reshape((ganTrain.shape[0], ganTrain.shape[1], 1))

In [0]:
# Optimizer
adam = Adam(lr=0.0002, beta_1=0.5)

d = Sequential()
# d.add(Embedding(vocab_size,embedding_dim, input_length=max_length))
# d.add(Reshape)
# d.add(LSTM(embedding_dim))
d.add(Dense(160))
d.add(Dense(280))
d.add(Dense(120))
d.add(Dense(60))
d.add(Dense(1))
d.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])


g = Sequential()
# g.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
g.add(LSTM(embedding_dim))
# g.add(Activation('elu'))
# g.add(Dense(embedding_dim))
# g.add(Round())
g.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

inputs = Input(shape=(max_length, 1))
hidden = g(inputs)
# print(type(hidden))
output = d(hidden)
gan = Model(inputs, output)
gan.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [0]:
BATCH_SIZE=2000
y = np.ones(BATCH_SIZE)
y = y.reshape((BATCH_SIZE,1))
gan.fit(train_data[:BATCH_SIZE], y)

In [0]:
t = g.predict(ganTrain[:1])
t

In [0]:
gan.summary()

In [0]:
def plot_loss(losses):
    """
    @losses.keys():
        0: loss
        1: accuracy
    """
    d_loss = [v[0] for v in losses["D"]]
    g_loss = [v[0] for v in losses["G"]]
    #d_acc = [v[1] for v in losses["D"]]
    #g_acc = [v[1] for v in losses["G"]]
    
    plt.figure(figsize=(10,8))
    plt.plot(d_loss, label="Discriminator loss")
    plt.plot(g_loss, label="Generator loss")
    #plt.plot(d_acc, label="Discriminator accuracy")
    #plt.plot(g_acc, label="Generator accuracy")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [0]:
BATCH_SIZE = 3000
epochs=10
plt_frq=1
losses = {"D":[], "G":[]}
batchCount = int(train_data.shape[0] / BATCH_SIZE)
print('Epochs:', epochs)
print('Batch size:', BATCH_SIZE)
print('Batches per epoch:', batchCount)

for e in range(1, epochs+1):
    if e == 1 or e%plt_frq == 0:
        print('-'*15, 'Epoch %d' % e, '-'*15)
    for i in range(batchCount):
        text_batch = train_data[np.random.randint(0, train_data.shape[0], size=BATCH_SIZE)]
        noise = ganTrain[np.random.randint(0, train_data.shape[0], size=BATCH_SIZE)]
        generated_text = g.predict(noise)
        # generated_text = generated_text.reshape((generated_text.shape[0], generated_text.shape[1], 1))
        x = np.concatenate((text_batch, generated_text))
        y = np.zeros(2*BATCH_SIZE)
        y[:BATCH_SIZE] = 1 #0.9  # One-sided label smoothing
        d.trainable = True
        d_loss = d.train_on_batch(x, y)

        gan_text_batch = ganTrain[np.random.randint(0, ganTrain.shape[0], size=BATCH_SIZE)]
        y2 = np.ones(BATCH_SIZE)
        d.trainable = False
        g_loss = gan.train_on_batch(gan_text_batch, y2)
    losses["D"].append(d_loss)
    losses["G"].append(g_loss)
plot_loss(losses)

### Second try:

In [0]:
train = digiData.comment
train = encode(train)
train = train.reshape((train.shape[0], train.shape[1], 1))
target2 = []
for i, v in digiData.iterrows():
    if v[13] == 0 or v[13] == 2:
        target2.append(0)
    else:
        target2.append(1)
target2 = np.array(target2)

In [0]:
discriminator = Sequential()
# discriminator.add(Embedding(vocab_size,embedding_dim, input_length=max_length))
discriminator.add(LSTM(max_length, input_shape=(max_length, 1)))
discriminator.add(Dense(100))
discriminator.add(Dense(1))
discriminator.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])

inputs = Input(shape=(max_length,))
output = discriminator(inputs)
model = Model(inputs, output)
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])


In [0]:
discriminator.summary()

In [0]:
model.fit(train, target2, epochs=10)

### Calculate subscription between "like comments" and "verylike comments"

In [0]:
# eshterak = {}
for i, sen1 in enumerate(like):
    for j, word1 in enumerate(sen1):
        if word1!=0:
            if (eshterak.get(str(word1))==None):
                eshterak[str(word1)] = 0
            if (eshterak.get(str(word1))==0):
                for n, sen2 in enumerate(verylike):
                    for m, word2 in enumerate(sen2):
                        if word2!=0 and word1==word2:
                            eshterak[str(word1)] = eshterak.get(str(word1))+1
                        if word2==0:
                            break
        else:
            break
    # print(sen1)
    if i%50 == 0:
        print(i, end=" ")